In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import sys
import json
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
# from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')


In [3]:
import sqlite3

#query=len(sys.argv)>1?sys.argv[1]:"php"

# Create a SQL connection to our SQLite database
con = sqlite3.connect("..\\OutliersAPP.db")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT * FROM Job'):
    # print(row)

# Be sure to close the connection
con.close()

IndentationError: expected an indented block (<ipython-input-3-2027d355388f>, line 15)

In [20]:

import pandas as pd
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("..\\OutliersAPP.db")
df = pd.read_sql_query("SELECT * FROM Job ", con)

# Verify that result of SQL query is stored in the dataframe
# print(df.head())

con.close()

Id  UserId       JobTitle ExperienceNeeded Salary   JobType  \
0   1      24             ff                f      f  PartTime   
1   2      24   web devolper           nurmal    500  PartTime   
2   3      24  game devolper             some   4000  FullTime   
3   4      24            php     programming    5000  PartTime   

  JobRequirements                     CareerLevel Vacancies  \
0             rrr       EntryLevel(freshGraduate)         f   
1            aaaa       EntryLevel(freshGraduate)         f   
2             fff  IntermediateorExperiencedLevel       aaa   
3         ffffddd       EntryLevel(freshGraduate)        aa   

                 JobDescription   City Country                      Created  \
0                           hyg      f       f  2020-05-11 23:55:51.7599029   
1  pig salary know css html php   alex   egypt  2020-06-08 17:52:57.1577042   
2               know c# and oop  cairo   egypt  2020-06-08 17:55:56.9349025   
3                      know php  cairo  

In [60]:
jobs_base_line = df

In [48]:
jobs_base_line['JobTitle'] = jobs_base_line['JobTitle'].fillna('')
jobs_base_line['JobDescription'] = jobs_base_line['JobDescription'].fillna('')


jobs_base_line['JobDescription'] = jobs_base_line['JobTitle'] + jobs_base_line['JobDescription']
jobs_base_line.head(20000)


,level_0,index,Id,UserId,JobTitle,ExperienceNeeded,Salary,JobType,JobRequirements,CareerLevel,Vacancies,JobDescription,City,Country,Created,JobRole
0,0,0,1,24,ff,f,f,PartTime,rrr,EntryLevel(freshGraduate),f,ffffhyg,f,f,2020-05-11 23:55:51.7599029,ff
1,1,1,2,24,web devolper,nurmal,500,PartTime,aaaa,EntryLevel(freshGraduate),f,web devolperweb devolperpig salary know css ht...,alex,egypt,2020-06-08 17:52:57.1577042,aaaa
2,2,2,3,24,game devolper,some,4000,FullTime,fff,IntermediateorExperiencedLevel,aaa,game devolpergame devolperknow c# and oop,cairo,egypt,2020-06-08 17:55:56.9349025,aaa
3,3,3,4,24,php,programming,5000,PartTime,ffffddd,EntryLevel(freshGraduate),aa,phpphpknow php,cairo,egypt,2020-06-08 18:01:47.1314275,aaa


In [61]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(jobs_base_line['JobDescription'])

In [62]:
tfidf_matrix.shape

(4, 21)

In [63]:

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [64]:
cosine_sim

array([[1.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.10806031],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.10806031, 0.        , 1.        ]])

In [65]:
jobs_base_line = jobs_base_line.reset_index()
titles = jobs_base_line['JobTitle']
indices = pd.Series(jobs_base_line.index, index=jobs_base_line['JobTitle'])
indices.head(2)

JobTitle
ff              0
web devolper    1
dtype: int64

In [66]:
def get_recommendations(title):
    idx = indices[title]
    #print (idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print (sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    job_indices = [i[0] for i in sim_scores]
    return titles.iloc[job_indices]

In [67]:
print(json.dumps(get_recommendations(query).head(10)))

0               ff
1     web devolper
2    game devolper
3              php
Name: JobTitle, dtype: object